In [1]:
import tensorflow as tf
import numpy as np
import scipy.sparse as sp
from scipy.sparse.linalg.eigen.arpack import eigsh
import scipy.sparse as sp
import pandas as pd
import pickle
from datetime import timedelta
import matplotlib.pyplot as plt
from scipy.stats.stats import pearsonr
#from tensorflow.contrib.rnn.python.ops import rnn_cell as RNNCell
from tensorflow.python.ops.rnn_cell_impl import _RNNCell as RNNCell
import collections
from tensorflow.contrib import rnn
import h5py
#from tensorflow.python.ops.rnn_cell_impl import _RNNCell 
import random

In [2]:
def normalize_adj(adj):
    """Symmetrically normalize adjacency matrix."""
    #adj[np.isnan(adj)] = 0.
    adj = tf.abs(adj)
    rowsum = tf.reduce_sum(adj, 1)# sum by row

    d_inv_sqrt = tf.pow(rowsum, -0.5)
   
    #d_inv_sqrt[np.isinf(d_inv_sqrt)] = 0.
    
    d_mat_inv_sqrt = tf.diag(d_inv_sqrt)

    return tf.matmul(tf.matmul(d_mat_inv_sqrt, adj), d_mat_inv_sqrt)

def masked_mae_tf(preds, labels, null_val=np.nan):
    """
    Accuracy with masking.
    :param preds:
    :param labels:
    :param null_val:
    :return:
    """
    if np.isnan(null_val):
        mask = ~tf.is_nan(labels)
    else:
        mask = tf.not_equal(labels, null_val)
    mask = tf.cast(mask, tf.float32)
    mask /= tf.reduce_mean(mask)
    mask = tf.where(tf.is_nan(mask), tf.zeros_like(mask), mask)
    loss = tf.abs(tf.subtract(preds, labels))
    loss = loss * mask
    loss = tf.where(tf.is_nan(loss), tf.zeros_like(loss), loss)
    return tf.reduce_mean(loss)

In [3]:
class StandardScaler:
    """
    Standard the input
    """

    def __init__(self, mean, std):
        self.mean = mean
        self.std = std

    def transform(self, data):
        return (data - self.mean) / self.std

    def inverse_transform(self, data):
        return (data * self.std) + self.mean


In [4]:
# install tables
raw_data = pd.read_hdf('../../data/METR-LA/metr-la.h5')

In [5]:
raw_data.shape

(34272, 207)

In [6]:
raw_data.head()

773869     767541     767542     717447     717446  \
2012-03-01 00:00:00  64.375000  67.625000  67.125000  61.500000  66.875000   
2012-03-01 00:05:00  62.666667  68.555556  65.444444  62.444444  64.444444   
2012-03-01 00:10:00  64.000000  63.750000  60.000000  59.000000  66.500000   
2012-03-01 00:15:00   0.000000   0.000000   0.000000   0.000000   0.000000   
2012-03-01 00:20:00   0.000000   0.000000   0.000000   0.000000   0.000000   

                        717445  773062  767620     737529     717816  ...  \
2012-03-01 00:00:00  68.750000  65.125  67.125  59.625000  62.750000  ...   
2012-03-01 00:05:00  68.111111  65.000  65.000  57.444444  63.333333  ...   
2012-03-01 00:10:00  66.250000  64.500  64.250  63.875000  65.375000  ...   
2012-03-01 00:15:00   0.000000   0.000   0.000   0.000000   0.000000  ...   
2012-03-01 00:20:00   0.000000   0.000   0.000   0.000000   0.000000  ...   

                        772167  769372     774204     769806  717590  \
2012-03-01 00:00:00  45.625000  65.500  64.500000  66.428571  66.875   
2012-03-01 00:05:00  50.666667  69.875  66.666667  58.555556  62.000   
2012-03-01 00:10:00  44.125000  69.000  56.500000  59.250000  68.125   
2012-03-01 00:15:00   0.000000   0.000   0.000000   0.000000   0.000   
2012-03-01 00:20:00   0.000000   0.000   0.000000   0.000000   0.000   

                        717592     717595     772168     718141  769373  
2012-03-01 00:00:00  59.375000  69.000000  59.250000  69.000000  61.875  
2012-03-01 00:05:00  61.111111  64.444444  55.888889  68.444444  62.875  
2012-03-01 00:10:00  62.500000  65.625000  61.375000  69.857143  62.000  
2012-03-01 00:15:00   0.000000   0.000000   0.000000   0.000000   0.000  
2012-03-01 00:20:00   0.000000   0.000000   0.000000   0.000000   0.000  

[5 rows x 207 columns]

In [7]:
# Create model
def gcn(x, weights, biases, batch_size, n_input, frequency,flag, n_output_vec, num = None):
    # Hidden layer with RELU activation
    
    output_list = tf.Variable(tf.zeros([n_output_vec,1]),dtype=tf.float32) #'Tensor' object does not support item assignment, cant build Ypre
    
    if flag == 1:
        rownum = batch_size
    elif flag == 2:
        rownum = num
    elif flag == 3:
        rownum = num
    
    for i in range(rownum):
        Xtem = x[i,:]
        #Xtem = tf.reshape(x[i,:], [n_input, frequency])
        #Xtem = tf.transpose(Xtem)
        #Atem = tf.convert_to_tensor(A_whole_final, dtype=np.float32)
        #Atem1 = tf.diag(tf.ones([n_input])) # Atem not palying any roles
        #Ytem = tf.reshape(Y[i,:], [n_input, 1])
        #Atem = tf.diag(tf.ones([n_input]))
        Atem1 = 0.5*(weights['A1'] + tf.transpose(weights['A1']))#+ Atem 
        Atem1 = normalize_adj(Atem1)
        #th = tf.constant(0.01, dtype=tf.float32)
        #where = tf.subtract(Atem1, th)
        #Atem1 = tf.nn.relu(where)
        
        Z1 = tf.matmul(Atem1, Xtem) #+ tf.matmul( tf.matmul(weights['A1'], weights['A1']), Xtem)
        #layer_1 = tf.matmul(Z1, weights['h1']) 
        layer_1 = tf.add(tf.matmul(Z1, weights['h1']), biases['b1'])
        layer_1 = tf.nn.relu(layer_1)
        
        Atem2 = 0.5*(weights['A2'] + tf.transpose(weights['A2']))#+ Atem 
        Atem2 = normalize_adj(Atem2)
        
        Z2 = tf.matmul(Atem2, layer_1)
        layer_2 = tf.add(tf.matmul(Z2, weights['h2']), biases['b2'])
        layer_2 = tf.nn.relu(layer_2)
        
        Atem3 = 0.5*(weights['A3'] + tf.transpose(weights['A3']))#+ Atem 
        Atem3 = normalize_adj(Atem3)
        Z3 = tf.matmul(Atem3, layer_2)
        layer_3 = tf.add(tf.matmul(Z3, weights['h3']), biases['b3'])
        layer_3 = tf.nn.relu(layer_3)
        
        #Atem4 = 0.5*(weights['A4'] + tf.transpose(weights['A4']))#+ Atem 
        #Atem4 = normalize_adj(Atem4)
        #Z4 = tf.matmul(Atem4, layer_3)
        #layer_4 = tf.add(tf.matmul(Z4, weights['h4']), biases['b4'])
        #layer_4 = tf.nn.relu(layer_4)
        
        # flattern
        #layer_3 = tf.reshape(layer_3, [1, 272*n_hidden_vec3])
        
        #F1 = tf.add(tf.matmul(layer_3, weights['f1']), biases['bf1'])
        #F1 = tf.nn.relu(F1)
        
        #F2 = tf.add(tf.matmul(F1, weights['f2']), biases['bf2'])
        #F2 = tf.nn.relu(F2)
        
        #F3 = tf.add(tf.matmul(F1, weights['f3']), biases['bf3'])
        #out_layer = tf.reshape(F3, [272, 1])

        # Output layer with linear activation
        Z4 = layer_3#tf.matmul(Atem1, layer_1)#tf.matmul(Atem, layer_3)
        out_layer = tf.add(tf.matmul(Z4, weights['out']), biases['bout'])
        #out_layer = tf.nn.relu(out_layer)
        # weather layer 1
        #x_wea_tem = tf.reshape(x_wea[i,:], [1, 9*frequency2]) # 1 by 126
        #layer_1_wea = tf.add(tf.matmul(x_wea_tem, weights['h1_wea']), biases['b1_wea'])
        #layer_1_wea = tf.nn.relu(layer_1_wea)
        
        #out_layer_wea = tf.add(tf.matmul(layer_1_wea, weights['out_wea']), biases['bout_wea'])
        #out_layer = tf.add(out_layer, tf.reshape(out_layer_wea, [272, 1]))
        
        #print (out_layer.get_shape())
        if i ==0:
            #print (out_layer.shape)
            tem = tf.reshape(out_layer, [1, n_output_vec])
            output_list = tem
        else:
            tem = tf.reshape(out_layer, [1, n_output_vec])
            output_list = tf.concat([output_list, tem], 0)
        
        #print (tf.reduce_mean(tf.pow(output_list-out_layer, 2)))
    
    #print (output_list.get_shape())
    #print ('here!!!!!!!!!!!!!!!!')
    #output_list = tf.transpose(output_list)
    
    #print (output_list.shape)
    
    #print (output_list.get_shape())
    
    return output_list

In [8]:
def gcn_corr_final(frequency, horizon, learning_rate, decay,batch_size, n_hidden_vec1,n_hidden_vec2,n_hidden_vec3,keep, early_stop_th,training_epochs, reg1, reg2):
    # set size
    #sn = 3 # station number
    X_whole = []
    Y_whole = []

    x_offsets = np.sort(
        # np.concatenate(([-week_size + 1, -day_size + 1], np.arange(-11, 1, 1)))
        np.concatenate((np.arange(-frequency+1, 1, 1),))
    )
    # Predict the next one hour
    y_offsets = np.sort(np.arange(1, 1+ horizon, 1))

    min_t = abs(min(x_offsets))
    max_t = abs(raw_data.shape[0] - abs(max(y_offsets)))  # Exclusive
    for t in range(min_t, max_t):
        x_t = raw_data.iloc[t + x_offsets, 0:sn].values.flatten('F')
        y_t = raw_data.iloc[t + y_offsets, 0:sn].values.flatten('F')
        X_whole.append(x_t)
        Y_whole.append(y_t)

    X_whole = np.stack(X_whole, axis=0)
    Y_whole = np.stack(Y_whole, axis=0)
    
    
    '''
    time_step = int(time_step) #

    #i = time_step
    #X_whole = np.zeros(shape = (raw_data.shape[0] - time_step, sn*time_step), dtype = np.float)
    #Y_whole = np.zeros(shape = (raw_data.shape[0] - time_step, sn), dtype = np.float)

    while i < raw_data.shape[0]:
        X_whole[i - time_step, ] = raw_data.iloc[(i - time_step):i, 0:sn].values.flatten('F') # 'F' flatten by column, default:flatten by row 0, 1, 2...7
        Y_whole[i - time_step, ] = raw_data.iloc[i, 0:sn]
        i = i + 1
    '''
    time_step = int(X_whole.shape[1] / sn)

    n_input = sn # station number
    n_input_vec = n_input * frequency # 207 * frequency
    n_A_vec = n_input * n_input
    n_output_vec = Y_whole.shape[1] # each row represent a result
    #print (n_output_vec)

    X_whole = np.reshape(X_whole, [X_whole.shape[0], sn, time_step])
    num_samples = X_whole.shape[0]
    num_test = round(num_samples * 0.2)
    num_train = round(num_samples * 0.7)
    num_val = num_samples - num_test - num_train
    #skip = skip1 + freq_max - time_step#time_step_max - time_step # to make sure the testing datasets are the same although the frequency could be different

    X_training = X_whole[:num_train, :]
    Y_training = Y_whole[:num_train, :]
    
    # shuffle
    perm = np.arange(X_training.shape[0])
    np.random.shuffle(perm)
    X_training = X_training[perm]
    Y_training = Y_training[perm]
    
    #print (type(X_training))
    #X_training = random.Random(6).shuffle(X_training)
    #Y_training = random.Random(6).shuffle(Y_training)

    X_val = X_whole[num_train:num_train+num_val, :]
    Y_val = Y_whole[num_train:num_train+num_val, :]
    #A_val = A_whole[0+training:0+training+validation, :]

    X_test = X_whole[-num_test:, :]
    Y_test = Y_whole[-num_test:, :]

    scaler = StandardScaler(mean=X_training.mean(), std=X_training.std())

    X_training = scaler.transform(X_training)
    Y_training = scaler.transform(Y_training)

    X_val = scaler.transform(X_val)
    Y_val = scaler.transform(Y_val)

    X_test = scaler.transform(X_test)
    Y_test = scaler.transform(Y_test)
    
    early_stop_th = int(early_stop_th)
    training_epochs = int(training_epochs)
    
    early_stop_k=0
    display_step = 1
    best_val = 10000
    traing_error = 0
    test_error = 0
    predic_res = []
    predic_step = []

    tf.reset_default_graph()

    keep_prob = tf.placeholder(tf.float32) #dropout (keep probability)
    
    
    batch_size = int(batch_size)
    n_hidden_vec1 = int(n_hidden_vec1)
    n_hidden_vec2 = int(n_hidden_vec2)
    n_hidden_vec3 = int(n_hidden_vec3)
    early_stop_th = int(early_stop_th)
    training_epochs = int(training_epochs)
    
    early_stop_k=0
    display_step = 1
    best_val = 10000
    traing_error = 0
    test_error = 0
    # Network Parameters

    #n_classes = 2 # MNIST total classes (0-9 digits) # n_classes is for classification only

    tf.reset_default_graph()

    # tf Graph input
    X = tf.placeholder(tf.float32, [None, sn, time_step]) # X is the input signal
    #X_weather = tf.placeholder(tf.float32, [None, 9 * frequency2]) # X_weather weather and holiday information (9 is the feature number)
    A = tf.placeholder(tf.float32, [None, n_A_vec]) # A is the normalized adj matrix
    oldA = tf.placeholder(tf.float32, [n_input, n_input])
    Y = tf.placeholder(tf.float32, [None, n_output_vec]) # y is the regression output

    #Xtem = tf.placeholder(tf.float32, [n_input, frequency]) # for each row of X, A, Y, it can be reshaped to Xtem, Atem, Ytem
    #Atem = tf.placeholder(tf.float32, [n_input, n_input]) # 
    #Ytem = tf.placeholder(tf.float32, [n_input, 1]) #

    #Ypre = tf.placeholder(tf.float32, [None, n_output_vec])

    keep_prob = tf.placeholder(tf.float32) #dropout (keep probability)

    # Store layers weight & bias
    weights = {
        'h1': tf.Variable(tf.random_normal([frequency, n_hidden_vec1])),
        'h2': tf.Variable(tf.random_normal([n_hidden_vec1, n_hidden_vec2])),
        'h3': tf.Variable(tf.random_normal([n_hidden_vec2, n_hidden_vec3])),
        #'h4': tf.Variable(tf.random_normal([n_hidden_vec3, n_hidden_vec4])),
        'out': tf.Variable(tf.random_normal([n_hidden_vec3, horizon])), # dont forget to change n_hidden_vec1 when add/delete layers
        #'f1': tf.Variable(tf.random_normal([272*n_hidden_vec3, 100])),
        #'f2': tf.Variable(tf.random_normal([50, 10])),
        #'f3': tf.Variable(tf.random_normal([100, 272])),
        'A1': tf.Variable(tf.random_normal([n_input,n_input])),
        'A2': tf.Variable(tf.random_normal([n_input,n_input])),
        'A3': tf.Variable(tf.random_normal([n_input,n_input])),
        #'A4': tf.Variable(tf.random_normal([n_input,n_input])),
        #'h1_wea': tf.Variable(tf.random_normal([9*frequency2, n_hidden_weather1])),
        #'out_wea': tf.Variable(tf.random_normal([n_hidden_weather1, n_input]))
    }
    biases = {
        'b1': tf.Variable(tf.random_normal([n_input,n_hidden_vec1])),# n_hidden_vec1])),# bias all the same??? dont forget to test
        'b2': tf.Variable(tf.random_normal([n_input, n_hidden_vec2])), #n_hidden_vec2])),
        'b3': tf.Variable(tf.random_normal([n_input, n_hidden_vec3])),#n_hidden_vec3])),
        #'b4': tf.Variable(tf.random_normal([n_input, n_hidden_vec4])),
        #'b1': tf.Variable(tf.random_normal([n_input,n_hidden_vec1])),# bias all the same??? dont forget to test
        #'b2': tf.Variable(tf.random_normal([n_input,n_hidden_vec2])),
        #'b3': tf.Variable(tf.random_normal([n_input,n_hidden_vec3])),
        #'bf1': tf.Variable(tf.random_normal([1, 100])), 
        #'bf2': tf.Variable(tf.random_normal([1, 10])), 
        #'bf3': tf.Variable(tf.random_normal([1, 272])), 
        'bout': tf.Variable(tf.random_normal([n_input, horizon])), 
        #'b1_wea': tf.Variable(tf.random_normal([1, n_hidden_weather1])), 
        #'bout_wea': tf.Variable(tf.random_normal([1, n_input])), 
    }

    #gcn_no_A
    #gcn
    # Construct model
    pred= gcn(X, weights, biases, batch_size,n_input, frequency, 1, n_output_vec)
    #print ('here!!!!!!!!!!!!!!!!')
    #set  negative in pred to 0s
    #Computes rectified linear: max(features, 0).
    #pred = tf.nn.relu(pred)
    # Define loss and optimizer
    # RMS for regression 
    # L2 regularization
    #cost = tf.reduce_mean(tf.pow(pred-Y, 2)) + reg1*tf.nn.l2_loss(weights['A1']) + reg2*tf.nn.l2_loss(weights['A2'])# + tf.nn.l2_loss(weights['A2']) + tf.nn.l2_loss(weights['A3'])))

    # L1 regular tf.reduce_sum(tf.abs(parameters))
    # mae
    
    pred = scaler.inverse_transform(pred)
    Y_true_tr = scaler.inverse_transform(Y)

    cost = masked_mae_tf(pred, Y_true_tr, 0)#+ reg1*tf.reduce_sum(tf.abs(weights['A1'])) +  reg2*tf.reduce_sum(tf.abs(weights['A2'])) + reg3*tf.reduce_sum(tf.abs(weights['A3']))#tf.reduce_mean(tf.abs(pred-Y_true_tr))# + reg1*tf.reduce_sum(tf.abs(weights['A1']))
    #cost = tf.reduce_mean(tf.pow(pred-Y, 2)) + reg1*tf.reduce_sum(tf.abs(weights['A1']))# + reg2*tf.reduce_sum(tf.abs(weights['A2']))
    pred_val= gcn(X, weights, biases, batch_size,n_input,frequency, 2, n_output_vec, num_val)
    #pred_val = tf.nn.relu(pred_val)
    #mae 
    pred_val = scaler.inverse_transform(pred_val)
    Y_true_val = scaler.inverse_transform(Y)
    cost_val = masked_mae_tf(pred_val, Y_true_val, 0)#tf.reduce_mean(tf.abs(pred_val-Y_true_val))
    #cost_val = tf.reduce_mean(tf.pow(pred_val-Y, 2))

    pred_tes= gcn(X, weights, biases, batch_size,n_input,frequency, 3, n_output_vec, num_test)
    #pred_tes = tf.nn.relu(pred_tes)
    # mae
    pred_tes = scaler.inverse_transform(pred_tes)
    Y_true_tes = scaler.inverse_transform(Y)
    '''
    predic_res_re = tf.reshape(pred_tes, [Y_test.shape[0], sn, horizon])
    test_Y_re = tf.reshape(Y_true_tes, [Y_test.shape[0], sn, horizon])
    
    cost_by_hor = []
    for hor_i in range(horizon):
        cost_by_hor.append(masked_mae_tf(predic_res_re[:, :, 0:hor_i], test_Y_re[:, :, 0:hor_i], 0))
    '''
    cost_tes = masked_mae_tf(pred_tes, Y_true_tes, 0)#tf.reduce_mean(tf.abs(pred_tes-Y_true_tes))
    #rmse
    #cost_tes = tf.reduce_mean(tf.pow(pred_tes-Y, 2))
    # cross-entropy for classification
    # cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=Y_train))
    # ratio = tf.abs(tf.reduce_sum(pred)-tf.reduce_sum(Y))/tf.reduce_sum(Y)
    #zero = 0
    #ratio = tf.reduce_mean(tf.divide(tf.where(tf.not_equal(Y, zero), np.abs(pred-Y), tf.zeros(Y.get_shape(), tf.float32)), tf.where(tf.not_equal(Y, zero), Y, tf.ones(Y.get_shape(), tf.float32))))
    optimizer = tf.train.RMSPropOptimizer(learning_rate, decay).minimize(cost)
    #optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

    # Initializing the variables
    init = tf.global_variables_initializer()
    saver = tf.train.Saver()

    #total_val_cost = []
    #total_val_ratio = []

    # learning start from 

    #index = daily_bike[(daily_bike['year'] == 2016) & (daily_bike['monthofyear'] == 1) & (daily_bike['dayofmonth'] == 1)].index.tolist()[0]
    #A_hat = normalize_adj(corr_matrix_trips)
    #print(A_hat)


    with tf.Session() as sess:
        sess.run(init)

        for epoch in range(training_epochs):

            avg_cost = 0.
            total_batch = int(num_train/batch_size)

            for i in range(total_batch):
                #print (Y_training[i*batch_size:(i+1)*batch_size,].size())
                _, c, preds, trueval = sess.run([optimizer, cost, pred, Y_true_tr], feed_dict={X: X_training[i*batch_size:(i+1)*batch_size,], 
                                                      Y: Y_training[i*batch_size:(i+1)*batch_size,], 
                                                              keep_prob: keep})
                #print (preds)
                #print (trueval)
                #print ("Epoch:", '%04d' % (epoch+1), "batch: ", i, "batch cost=", \
                #    "{:.9f}".format(c))
                #print ('here!!!!!!!!!!!!!!!!')
                avg_cost += c / total_batch 
                #Display logs per epoch step
            if epoch % display_step == 0:
                print ("Epoch:", '%04d' % (epoch+1), "Training MAE=", \
                    "{:.9f}".format(avg_cost)) #np.sqrt(avg_cost)
            # validation
            c_val = sess.run([cost_val], feed_dict={X: X_val, Y: Y_val,  keep_prob:1})
            print("Validation MAE: ", c_val[0])

            c_tes, pred_tes1, A1, Y_true = sess.run([cost_tes, pred_tes, weights['A1'], Y_true_tes], feed_dict={X: X_test,Y: Y_test, keep_prob: 1})
            print("Test MAE: ", c_tes)
            #print("predic step: ", cost_by_hor)

            if c_val[0] < best_val:
                best_val = c_val[0]
                #saver.save(sess, './bikesharing_graph_2_th_point1')
                test_error = c_tes
                traing_error = avg_cost#np.sqrt(avg_cost)
                early_stop_k = 0 # reset to 0
                #print (pred_tes1)
                predic_res = pred_tes1
                #predic_step = cost_by_hor

            # early stopping
            if c_val[0] >= best_val:
                early_stop_k += 1

            # threshold
            if early_stop_k == early_stop_th:
              #  print ("early stopping...")
                break
            

        print("epoch is ", epoch)
        print("training error is ", traing_error)
        print("Optimization Finished! the lowest validation MAE is ", best_val)#(np.sqrt(best_val)))
        print("The test MAE is ", test_error)#(np.sqrt(test_error)))
    
    test_Y = Y_test
    test_error = np.sqrt(test_error)
    return -best_val, predic_res,Y_true,test_error, A1#, predic_step#, A2

In [34]:
# change hidden number
# change batch_size
# set size
#from bayes_opt import BayesianOptimization
import datetime
#freq_max = 12
#time_step = 12
learning_rate = 0.01
decay = 0.9 
batch_size = 50
num_hidden = 64 # number of hiddent units in LSTM Cell
early_stop_th = 500
training_epochs = 1000
keep = 1#0.2
#time_step_max = 10
feat_in = 1
feat_out = 1
sn = 207 # station num
#test = 2000 
reg1 = 0.05#0.05
reg2 = 0#0.1
frequency = 12
horizon = 12
n_hidden_vec1 = 10
n_hidden_vec2 = 10
n_hidden_vec3 = 10
#All_pred = np.empty([2000, 207])
#All_Y = np.empty([2000, 207])

#24*90
#step = 0
#gap = 100
#training = 0.7
#validation = 0.1
#test = 0.2

#gcn_corr_eval(7, 0.01, 0.5, 100, 0.4, 10, 5, 5, 0.2, 50, 500)



rep = 1 # repeating times

#total_sn = 0
#num_iter = 50
#init_points = 200


# stdbscan
#spatial_threshold = 300
#temporal_threshold = 300
#min_neighbors = 1 # number of neighbor

#frequency2 = skip1 + freq_max + training

#while step < 2000:

#hourly_bike_cluster = hourly_bike
best = -10000
pre_best = []
test_Y_best = []
test_error_best = 1000
A1_best = []
# A2_best = []
for i in range(rep):
    a = datetime.datetime.now()
    val_error, predic_res, test_Y,test_error, A1=gcn_corr_final(frequency, horizon, learning_rate, decay, batch_size, n_hidden_vec1,
                                                                n_hidden_vec2, n_hidden_vec3, keep, early_stop_th, training_epochs, reg1, reg2)
    #val_error, predic_res, test_Y,test_error=gcn_corr_final(a['frequency'], a['learning_rate'], a['decay'], a['batch_size'], a['n_hidden_vec1'], a['n_hidden_vec2'], a['n_hidden_vec3'], a['keep'], a['early_stop_th'], a['training_epochs'], a['reg'])
    #print ("finished A running: ", i)
    b = datetime.datetime.now()
    print(b-a)
    if val_error > best:
        best = val_error
        pre_best = predic_res
        test_Y_best = test_Y
        test_error_best = test_error
        #A1_best = A1
       # A2_best = A2

    #val_error, predic_res, test_Y,test_error=gcn_corr_final(24, 0.02, 0.2, 100, 5, 10, 10, 0.8, 50, 500)
    #All_pred = pre_best
    #All_Y  = test_Y_best

    #total_sn = total_sn + sn

    #total_error = np.sqrt(np.mean((All_pred[0:(step+gap),0:total_sn] - All_Y[0:(step+gap),0:total_sn])**2))

    #print ("!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!the cluster now is: ", c)
    #print ("!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!the val error of this cluster now is: ", best)
    #print ("!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!the test error by this cluster now is: ", total_error)
    #step = step + gap
    #skip1 = skip1 + gap
    
    #np.savetxt("prediction_300_300.csv", All_pred, delimiter = ',')
    #np.savetxt("prediction_Y_300_300.csv", All_Y, delimiter = ',')
    

KeyboardInterrupt: 

In [ ]:
# increas batch size from 100 to 1000: validation error 39.XX at about 100 epochs
# batch size to 50: validation error 6.XX at about 100 epochs
# add one more gcn layer: 8.XX
# increase learning rate to 0.01, batch size is 100: 10.XX
# decrease learning rate to 0.005, batch size is 100: 10.XX
# decrease learning rate to 0.005, batch size is 50, two hidden levels (same matrix):
    # 6.XX at 100th epoch
# decrease learning rate to 0.005, batch size is 50, one hidden level, hidden 40:
    # 6.XX at 100th epoch
# decrease learning rate to 0.005, batch size is 50, one hidden level, hidden 20:
    # drop much faster
    # 6.XX can be get, but kind of overfitting at 100th epoch?
# decrease learning rate to 0.005, batch size is 50, one hidden level, hidden 10:
    # drop much faster
    #  at 100th epoch
# learning rate to 0.01, batch size is 50, one hidden level, hidden 10, 
    #reduce prediction horizon from 12 to 1, normalize the data:
    # val: 3.16

# increase sample, learning rate to 0.01, batch size is 50, one hidden level, hidden 10, 
    #reduce prediction horizon from 12 to 1, normalize the data, normalize the symmetric adjacency matrix:
    # train: 3.24, val: 3.42
    
# increase sample, learning rate to 0.01, batch size is 50, one hidden level, hidden 10, 
    #reduce prediction horizon from 12 to 1, normalize the data, normalize the symmetric adjacency matrix, 
    # train: 3.08, val: 3.26
    
# increase sample, learning rate to 0.01, batch size is 50, one hidden level, hidden 10, 
    #reduce prediction horizon from 12 to 1, normalize the data, normalize the symmetric adjacency matrix, 
    # remove "0" ground truths as the paper
    # train: 2.82, val: 2.75
    
# increase sample, learning rate to 0.01, batch size is 50, one hidden level, hidden 10, 
    #prediction horizon 12, normalize the data, normalize the symmetric adjacency matrix, 
    # remove "0" ground truths as the paper, one layer
    # train: 4.08, val: 3.99, test: 4.29 
    
# increase sample, learning rate to 0.01, batch size is 50, one hidden level, hidden 10, 
    #prediction horizon 12, normalize the data, normalize the symmetric adjacency matrix, 
    # remove "0" ground truths as the paper, two layers (same adjmatrix)
    # train: 4.10, val: 4.19, test: 4.54 
    
# increase sample, learning rate to 0.01, batch size is 50, one hidden level, hidden 10, 
    #prediction horizon 12, normalize the data, normalize the symmetric adjacency matrix, 
    # remove "0" ground truths as the paper, two layers (different adjmatrix)
    # train: 3.52, val: 3.62, test: 4.03
    
# increase sample, learning rate to 0.01, batch size is 50, one hidden level, hidden 10, 
    #prediction horizon 12, normalize the data, normalize the symmetric adjacency matrix, 
    # remove "0" ground truths as the paper, two layers (different adjmatrix), bias set as a vector of same length
    # train: 3.53, val: 3.58, test: 3.91 
    
# increase sample, learning rate to 0.01, batch size is 50, one hidden level, hidden 10, 
    #prediction horizon 12, normalize the data, normalize the symmetric adjacency matrix, 
    # remove "0" ground truths as the paper, three layers (three adjmatrix), bias set as a vector of same length
    # batch from 50 to 100
    # train: 3.36, val: 3.42, test: 3.82
    
# increase sample, learning rate to 0.01, batch size is 50, one hidden level, hidden 10, 
    #prediction horizon 12, normalize the data, normalize the symmetric adjacency matrix, 
    # remove "0" ground truths as the paper, three layers (three adjmatrix), bias set as a vector of same length
    # batch from 50 to 100
    # last hidden size 10 to 20
    # train: 3.33, val: 3.38, test: 3.71
    # train: 3.24, val: 3.31, test: 3.66
    # train: 3.22, val: 3.29, test: 3.66
    
# increase sample, learning rate to 0.01, batch size is 50, one hidden level, hidden 10, 
    #prediction horizon 12, normalize the data, normalize the symmetric adjacency matrix, 
    # remove "0" ground truths as the paper, three layers (three adjmatrix), bias set as a vector of same length
    # batch from 50 to 100
    # three hidden size 5, 5, 10
    # train: 3.5, val: 3.48, test: 3.82 
    
# increase sample, learning rate to 0.01, batch size is 50, one hidden level, hidden 10, 
    #prediction horizon 12, normalize the data, normalize the symmetric adjacency matrix, 
    # remove "0" ground truths as the paper, three layers (three adjmatrix), bias set as a vector of same length
    # batch from 50 to 100
    # three hidden size 10, 10, 20
    # learning rate: 0.005
    # train: 3.23, val: 3.34, test: 3.74
    
# increase sample, learning rate to 0.01, batch size is 50, one hidden level, hidden 10, 
    #prediction horizon 12, normalize the data, normalize the symmetric adjacency matrix, 
    # remove "0" ground truths as the paper, three layers (three adjmatrix), bias set as a vector of same length
    # batch from 50 to 200
    # three hidden size 10, 10, 20
    # learning rate: 0.01
    # shuffle
    # train: 3.19, val: 3.30, test: 3.67
    
# increase sample, learning rate to 0.01, batch size is 50, one hidden level, hidden 10, 
    #prediction horizon 12, normalize the data, normalize the symmetric adjacency matrix, 
    # remove "0" ground truths as the paper, three layers (three adjmatrix), bias set as a vector of same length
    # batch from 50 to 100
    # three hidden size 10, 10, 20
    # learning rate: 0.01
    # shuffle
    # train: 3.15, val: 3.26, test: 3.61
    
# increase sample, learning rate to 0.01, batch size is 50, one hidden level, hidden 10, 
    #prediction horizon 12, normalize the data, normalize the symmetric adjacency matrix, 
    # remove "0" ground truths as the paper, three layers (three adjmatrix), bias set as a vector of same length
    # batch from 50 to 100
    # three hidden size 10, 10, 20
    # learning rate: 0.01
    # shuffle
    # fully freedom adjacency matrix (not symmetric )
    # train: , val: , test: 
    # training cost around 6.15, not decreasing anymore
    
    
# increase sample, learning rate to 0.01, batch size is 50, one hidden level, hidden 10, 
    #prediction horizon 12, normalize the data, normalize the symmetric adjacency matrix, 
    # remove "0" ground truths as the paper, three layers (three adjmatrix), bias set as a vector of same length
    # batch from 50 to 100
    # three hidden size 10, 10, 20
    # learning rate: 0.005
    # change gradient decent algorithm
    # train: 3.19, val: 3.28, test: 3.67

# increase sample, learning rate to 0.01, batch size is 50, one hidden level, hidden 10, 
    #prediction horizon 12, normalize the data, normalize the symmetric adjacency matrix, 
    # remove "0" ground truths as the paper, three layers (three adjmatrix), bias set as a vector of same length
    # batch from 50 to 100
    # three hidden size 10, 10, 20
    # learning rate: 0.005
    # change gradient decent algorithm
    # keep: 0.8
    # train: 3.30, val: 3.36, test: 3.78
    
# increase sample, learning rate to 0.01, batch size is 50, one hidden level, hidden 10, 
    #prediction horizon 12, normalize the data, normalize the symmetric adjacency matrix, 
    # remove "0" ground truths as the paper, three layers (three adjmatrix), bias set as a vector of same length
    # batch from 50 to 200
    # three hidden size 10, 10, 20
    # learning rate: 0.01
    # change gradient decent algorithm
    # keep: 1
    # train: 3.15, val: 3.27, test: 3.63
    
# increase sample, learning rate to 0.01, batch size is 50, one hidden level, hidden 10, 
    #prediction horizon 12, normalize the data, normalize the symmetric adjacency matrix, 
    # remove "0" ground truths as the paper, three layers (three adjmatrix), bias set as a vector of same length
    # batch from 50 to 200
    # three hidden size 10, 10, 20
    # learning rate: 0.01
    # change gradient decent algorithm
    # keep: 1
    # bias not the same size
    # decreasing slow

    
# increase sample, learning rate to 0.01, batch size is 50, one hidden level, hidden 10, 
    #prediction horizon 12, normalize the data, normalize the symmetric adjacency matrix, 
    # remove "0" ground truths as the paper, three layers (three adjmatrix), bias set as a vector of same length
    # batch size 200
    # second size 10 to 10
    # last hidden size 10 to 30
    # set the same size may lead to indecreasing cost (need to increase batch size)
    # train: 3.05, val: 3.22, test: 3.73

# batch size 150 really slow


In [13]:
# change hidden number
# change batch_size
# set size
#from bayes_opt import BayesianOptimization
import datetime
#freq_max = 12
#time_step = 12
learning_rate = 0.01
decay = 0.9 
batch_size = 100
num_hidden = 64 # number of hiddent units in LSTM Cell
early_stop_th = 500
training_epochs = 1000
keep = 1#0.2
#time_step_max = 10
feat_in = 1
feat_out = 1
sn = 207 # station num
#test = 2000 
reg1 = 0.05#0.05
reg2 = 0#0.1
frequency = 12
horizon = 12
n_hidden_vec1 = 10
n_hidden_vec2 = 10
n_hidden_vec3 = 20
#All_pred = np.empty([2000, 207])
#All_Y = np.empty([2000, 207])

#24*90
#step = 0
#gap = 100
#training = 0.7
#validation = 0.1
#test = 0.2

#gcn_corr_eval(7, 0.01, 0.5, 100, 0.4, 10, 5, 5, 0.2, 50, 500)



rep = 1 # repeating times

#total_sn = 0
#num_iter = 50
#init_points = 200


# stdbscan
#spatial_threshold = 300
#temporal_threshold = 300
#min_neighbors = 1 # number of neighbor

#frequency2 = skip1 + freq_max + training

#while step < 2000:

#hourly_bike_cluster = hourly_bike
best = -10000
pre_best = []
test_Y_best = []
test_error_best = 1000
A1_best = []
# A2_best = []
for i in range(rep):
    a = datetime.datetime.now()
    val_error, predic_res, test_Y,test_error, A1=gcn_corr_final(frequency, horizon, learning_rate, decay, batch_size, n_hidden_vec1,
                                                                n_hidden_vec2, n_hidden_vec3, keep, early_stop_th, training_epochs, reg1, reg2)
    #val_error, predic_res, test_Y,test_error=gcn_corr_final(a['frequency'], a['learning_rate'], a['decay'], a['batch_size'], a['n_hidden_vec1'], a['n_hidden_vec2'], a['n_hidden_vec3'], a['keep'], a['early_stop_th'], a['training_epochs'], a['reg'])
    #print ("finished A running: ", i)
    b = datetime.datetime.now()
    print(b-a)
    if val_error > best:
        best = val_error
        pre_best = predic_res
        test_Y_best = test_Y
        test_error_best = test_error
        #A1_best = A1
       # A2_best = A2

    #val_error, predic_res, test_Y,test_error=gcn_corr_final(24, 0.02, 0.2, 100, 5, 10, 10, 0.8, 50, 500)
    #All_pred = pre_best
    #All_Y  = test_Y_best

    #total_sn = total_sn + sn

    #total_error = np.sqrt(np.mean((All_pred[0:(step+gap),0:total_sn] - All_Y[0:(step+gap),0:total_sn])**2))

    #print ("!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!the cluster now is: ", c)
    #print ("!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!the val error of this cluster now is: ", best)
    #print ("!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!the test error by this cluster now is: ", total_error)
    #step = step + gap
    #skip1 = skip1 + gap
    
    #np.savetxt("prediction_300_300.csv", All_pred, delimiter = ',')
    #np.savetxt("prediction_Y_300_300.csv", All_Y, delimiter = ',')
    

Epoch: 0001 Training MAE= 34.149538888
Validation MAE:  7.796924
Test MAE:  8.399293
Epoch: 0002 Training MAE= 6.981501669
Validation MAE:  6.0433497
Test MAE:  6.6129375
Epoch: 0003 Training MAE= 5.902498460
Validation MAE:  5.4331007
Test MAE:  5.943236
Epoch: 0004 Training MAE= 5.490435540
Validation MAE:  5.2061534
Test MAE:  5.638655
Epoch: 0005 Training MAE= 5.237063695
Validation MAE:  4.9617186
Test MAE:  5.344722
Epoch: 0006 Training MAE= 4.993687981
Validation MAE:  4.709289
Test MAE:  5.0467653
Epoch: 0007 Training MAE= 4.775287294
Validation MAE:  4.56508
Test MAE:  4.898537
Epoch: 0008 Training MAE= 4.631835074
Validation MAE:  4.4564033
Test MAE:  4.8050075
Epoch: 0009 Training MAE= 4.538600935
Validation MAE:  4.3755865
Test MAE:  4.7298956
Epoch: 0010 Training MAE= 4.458476225
Validation MAE:  4.315274
Test MAE:  4.6800356
Epoch: 0011 Training MAE= 4.391734454
Validation MAE:  4.2242646
Test MAE:  4.5996203
Epoch: 0012 Training MAE= 4.317388269
Validation MAE:  4.223186

Validation MAE:  3.4384186
Test MAE:  3.8000746
Epoch: 0098 Training MAE= 3.344806828
Validation MAE:  3.4870515
Test MAE:  3.8265274
Epoch: 0099 Training MAE= 3.340028051
Validation MAE:  3.4188375
Test MAE:  3.752637
Epoch: 0100 Training MAE= 3.335190281
Validation MAE:  3.4224482
Test MAE:  3.7695737
Epoch: 0101 Training MAE= 3.345857009
Validation MAE:  3.4726522
Test MAE:  3.8106017
Epoch: 0102 Training MAE= 3.334791368
Validation MAE:  3.4361434
Test MAE:  3.7708452
Epoch: 0103 Training MAE= 3.346601647
Validation MAE:  3.470183
Test MAE:  3.8329222
Epoch: 0104 Training MAE= 3.332912395
Validation MAE:  3.4495451
Test MAE:  3.8373165
Epoch: 0105 Training MAE= 3.333688368
Validation MAE:  3.4623988
Test MAE:  3.8187032
Epoch: 0106 Training MAE= 3.319697913
Validation MAE:  3.4417882
Test MAE:  3.8053641
Epoch: 0107 Training MAE= 3.333643535
Validation MAE:  3.3870099
Test MAE:  3.719692
Epoch: 0108 Training MAE= 3.327552820
Validation MAE:  3.4659777
Test MAE:  3.8046746
Epoch: 01

Validation MAE:  3.4589598
Test MAE:  3.8794703
Epoch: 0194 Training MAE= 3.218390151
Validation MAE:  3.2889404
Test MAE:  3.6470928
Epoch: 0195 Training MAE= 3.217002961
Validation MAE:  3.3000588
Test MAE:  3.6636727
Epoch: 0196 Training MAE= 3.214244029
Validation MAE:  3.3265543
Test MAE:  3.692969
Epoch: 0197 Training MAE= 3.211218075
Validation MAE:  3.4279714
Test MAE:  3.7947073
Epoch: 0198 Training MAE= 3.217877952
Validation MAE:  3.3358924
Test MAE:  3.7475688
Epoch: 0199 Training MAE= 3.234875079
Validation MAE:  3.3253803
Test MAE:  3.735373
Epoch: 0200 Training MAE= 3.212779349
Validation MAE:  3.2871375
Test MAE:  3.6567311
Epoch: 0201 Training MAE= 3.207691577
Validation MAE:  3.300047
Test MAE:  3.7046406
Epoch: 0202 Training MAE= 3.209427016
Validation MAE:  3.2954557
Test MAE:  3.6624522
Epoch: 0203 Training MAE= 3.207038467
Validation MAE:  3.4729476
Test MAE:  3.9225855
Epoch: 0204 Training MAE= 3.210953010
Validation MAE:  3.3116622
Test MAE:  3.7243967
Epoch: 02

KeyboardInterrupt: 

In [9]:
# change hidden number
# change batch_size
# set size
#from bayes_opt import BayesianOptimization
import datetime
#freq_max = 12
#time_step = 12
learning_rate = 0.01
decay = 0.9 
batch_size = 100
num_hidden = 64 # number of hiddent units in LSTM Cell
early_stop_th = 150
training_epochs = 150
keep = 1#0.2
#time_step_max = 10
feat_in = 1
feat_out = 1
sn = 207 # station num
#test = 2000 
reg1 = 0.05#0.05
reg2 = 0.05#0.1
reg3 = 0.05
frequency = 12
horizon = 12
n_hidden_vec1 = 10
n_hidden_vec2 = 10
n_hidden_vec3 = 20
n_hidden_vec4 = 10
#All_pred = np.empty([2000, 207])
#All_Y = np.empty([2000, 207])

#24*90
#step = 0
#gap = 100
#training = 0.7
#validation = 0.1
#test = 0.2

#gcn_corr_eval(7, 0.01, 0.5, 100, 0.4, 10, 5, 5, 0.2, 50, 500)



rep = 1 # repeating times

#total_sn = 0
#num_iter = 50
#init_points = 200


# stdbscan
#spatial_threshold = 300
#temporal_threshold = 300
#min_neighbors = 1 # number of neighbor

#frequency2 = skip1 + freq_max + training

#while step < 2000:

#hourly_bike_cluster = hourly_bike
best = -10000
pre_best = []
test_Y_best = []
test_error_best = 1000
A1_best = []
# A2_best = []
for i in range(rep):
    a = datetime.datetime.now()
    val_error, predic_res, test_Y,test_error, A1=gcn_corr_final(frequency, horizon, learning_rate, decay, batch_size, n_hidden_vec1,
                                                                n_hidden_vec2, n_hidden_vec3, keep, early_stop_th, training_epochs, reg1, reg2)
    #val_error, predic_res, test_Y,test_error=gcn_corr_final(a['frequency'], a['learning_rate'], a['decay'], a['batch_size'], a['n_hidden_vec1'], a['n_hidden_vec2'], a['n_hidden_vec3'], a['keep'], a['early_stop_th'], a['training_epochs'], a['reg'])
    #print ("finished A running: ", i)
    b = datetime.datetime.now()
    print(b-a)
    if val_error > best:
        best = val_error
        pre_best = predic_res
        test_Y_best = test_Y
        test_error_best = test_error
        #A1_best = A1
       # A2_best = A2

    #val_error, predic_res, test_Y,test_error=gcn_corr_final(24, 0.02, 0.2, 100, 5, 10, 10, 0.8, 50, 500)
    All_pred = pre_best
    All_Y  = test_Y_best

    #total_sn = total_sn + sn

    #total_error = np.sqrt(np.mean((All_pred[0:(step+gap),0:total_sn] - All_Y[0:(step+gap),0:total_sn])**2))

    #print ("!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!the cluster now is: ", c)
    #print ("!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!the val error of this cluster now is: ", best)
    #print ("!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!the test error by this cluster now is: ", total_error)
    #step = step + gap
    #skip1 = skip1 + gap
    
    #np.savetxt("prediction_2.csv", All_pred, delimiter = ',')
    #np.savetxt("prediction_Y_2.csv", All_Y, delimiter = ',')
    

Epoch: 0001 Training MAE= 27.551192343
Validation MAE:  7.177435
Test MAE:  7.466093
Epoch: 0002 Training MAE= 5.827730474
Validation MAE:  5.4311657
Test MAE:  5.8596067
Epoch: 0003 Training MAE= 5.212761468
Validation MAE:  5.0457287
Test MAE:  5.4020905
Epoch: 0004 Training MAE= 4.836488684
Validation MAE:  4.6494937
Test MAE:  5.0468764
Epoch: 0005 Training MAE= 4.664643319
Validation MAE:  4.5718207
Test MAE:  4.9534726
Epoch: 0006 Training MAE= 4.547515751
Validation MAE:  4.3695397
Test MAE:  4.778799
Epoch: 0007 Training MAE= 4.445308285
Validation MAE:  4.288931
Test MAE:  4.6967006
Epoch: 0008 Training MAE= 4.357100605
Validation MAE:  4.295091
Test MAE:  4.689468
Epoch: 0009 Training MAE= 4.256229830
Validation MAE:  4.1507673
Test MAE:  4.529448
Epoch: 0010 Training MAE= 4.176087137
Validation MAE:  4.082223
Test MAE:  4.449691
Epoch: 0011 Training MAE= 4.111285742
Validation MAE:  4.016917
Test MAE:  4.379396
Epoch: 0012 Training MAE= 4.039872332
Validation MAE:  3.9421122

KeyboardInterrupt: 

In [ ]:
best

In [ ]:
All_pred_tf = tf.convert_to_tensor(
    All_pred,
    dtype=np.float32,
    name=None,
    preferred_dtype=None
)

#All_Y = scaler.inverse_transform(All_Y)

All_Y_tf =  tf.convert_to_tensor(
    All_Y,
    dtype=np.float32,
    name=None,
    preferred_dtype=None
)

predic_res_re = tf.reshape(All_pred_tf, [All_pred.shape[0], sn, horizon])

test_Y_re = tf.reshape(All_Y_tf, [All_pred.shape[0], sn, horizon])

cost_by_hor = []
with tf.Session() as sess:  
    for hor_i in range(horizon):
        cost_by_hor.append(masked_mae_tf(predic_res_re[:, :, 0:hor_i+1], test_Y_re[:, :, 0:hor_i+1], 0).eval())

In [ ]:
cost_by_hor

In [76]:
All_Y

array([[65.25    , 66.625   , 63.555557, ..., 63.875   , 65.      ,
        63.88889 ],
       [66.625   , 63.555557, 62.625   , ..., 65.      , 63.88889 ,
        65.375   ],
       [63.555557, 62.625   , 65.875   , ..., 63.88889 , 65.375   ,
        64.5     ],
       ...,
       [65.375   , 63.5     , 66.375   , ..., 62.333336, 62.      ,
        61.222225],
       [63.5     , 66.375   , 65.55556 , ..., 62.      , 61.222225,
        63.5     ],
       [66.375   , 65.55556 , 66.375   , ..., 61.222225, 63.5     ,
        61.77778 ]], dtype=float32)

In [80]:
Y_test

array([[65.25      , 66.625     , 63.55555556, ..., 63.875     ,
        65.        , 63.88888889],
       [66.625     , 63.55555556, 62.625     , ..., 65.        ,
        63.88888889, 65.375     ],
       [63.55555556, 62.625     , 65.875     , ..., 63.88888889,
        65.375     , 64.5       ],
       ...,
       [65.375     , 63.5       , 66.375     , ..., 62.33333333,
        62.        , 61.22222222],
       [63.5       , 66.375     , 65.55555556, ..., 62.        ,
        61.22222222, 63.5       ],
       [66.375     , 65.55555556, 66.375     , ..., 61.22222222,
        63.5       , 61.77777778]])

In [79]:
All_Y.shape

(6850, 2484)

In [78]:
np.count_nonzero(All_Y)

14955041

In [77]:
X_whole = []
Y_whole = []

x_offsets = np.sort(
    # np.concatenate(([-week_size + 1, -day_size + 1], np.arange(-11, 1, 1)))
    np.concatenate((np.arange(-frequency+1, 1, 1),))
)
# Predict the next one hour
y_offsets = np.sort(np.arange(1, 1+ horizon, 1))

min_t = abs(min(x_offsets))
max_t = abs(raw_data.shape[0] - abs(max(y_offsets)))  # Exclusive
for t in range(min_t, max_t):
    x_t = raw_data.iloc[t + x_offsets, 0:sn].values.flatten('F')
    y_t = raw_data.iloc[t + y_offsets, 0:sn].values.flatten('F')
    X_whole.append(x_t)
    Y_whole.append(y_t)

X_whole = np.stack(X_whole, axis=0)
Y_whole = np.stack(Y_whole, axis=0)
'''
time_step = int(time_step) #

#i = time_step
#X_whole = np.zeros(shape = (raw_data.shape[0] - time_step, sn*time_step), dtype = np.float)
#Y_whole = np.zeros(shape = (raw_data.shape[0] - time_step, sn), dtype = np.float)

while i < raw_data.shape[0]:
    X_whole[i - time_step, ] = raw_data.iloc[(i - time_step):i, 0:sn].values.flatten('F') # 'F' flatten by column, default:flatten by row 0, 1, 2...7
    Y_whole[i - time_step, ] = raw_data.iloc[i, 0:sn]
    i = i + 1
'''
time_step = int(X_whole.shape[1] / sn)

n_input = sn # station number
n_input_vec = n_input * frequency # 207 * frequency
n_A_vec = n_input * n_input
n_output_vec = Y_whole.shape[1] # each row represent a result
#print (n_output_vec)

X_whole = np.reshape(X_whole, [X_whole.shape[0], sn, time_step])
num_samples = X_whole.shape[0]
num_test = round(num_samples * 0.2)
num_train = round(num_samples * 0.7)
num_val = num_samples - num_test - num_train
#skip = skip1 + freq_max - time_step#time_step_max - time_step # to make sure the testing datasets are the same although the frequency could be different

X_training = X_whole[:num_train, :]
Y_training = Y_whole[:num_train, :]

X_val = X_whole[num_train:num_train+num_val, :]
Y_val = Y_whole[num_train:num_train+num_val, :]
#A_val = A_whole[0+training:0+training+validation, :]

X_test = X_whole[-num_test:, :]
Y_test = Y_whole[-num_test:, :]



In [29]:
predic_res_re = predic_res.reshape(6850, 207, 12)
test_Y = scaler.inverse_transform(test_Y)
test_Y_re = test_Y.reshape(6850, 207, 12)



In [31]:
np.mean(abs(predic_res_re - test_Y_re), 2)

array([[1.65659099, 3.55353564, 2.21672849, ..., 0.99873041, 1.91440165,
        0.68093505],
       [1.7329778 , 3.21970576, 1.89514316, ..., 1.01460432, 1.6958958 ,
        0.6493283 ],
       [1.6444866 , 3.79785365, 1.75995028, ..., 0.87463315, 1.85563434,
        0.59415005],
       ...,
       [1.58595717, 1.1323835 , 1.08822678, ..., 1.73657514, 1.28728091,
        2.01705333],
       [1.6801099 , 1.38724151, 1.18594407, ..., 1.72045296, 1.2552597 ,
        2.0303307 ],
       [1.56032043, 1.41434014, 1.20248085, ..., 1.6380372 , 1.1316544 ,
        2.08039327]])

In [39]:
for horizon_i in range(12):
    y_truth = test_Y_re[:, :, horizon_i]
    y_pred = predic_res_re[:, :, horizon_i]


    tem = abs(y_pred - y_truth)
    res = []
    for i in range(y_truth.shape[0]):
        for j in range(y_truth.shape[1]):
            if y_truth[i, j] != 0:
                res.append(tem[i, j])
    print (np.mean(res))

10.201725449714056
10.207749365110994
10.234614605130048
10.285593437729197
10.347051168324938
10.411077811985333
10.47475043743939
10.54087956149636
10.603807081608199
10.663507683205278
10.724685387314283
10.791137661290614


In [37]:
y_truth.shape

(6850, 207)

In [42]:
len(res)

1417950